<a href="https://colab.research.google.com/github/yjyjy131/Study_Deep_Learning/blob/main/Pytorch_Basic/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Linear Regression

In [1]:
import torch
import torch.nn as nn
from matplotlib import pyplot as plt

In [3]:
# unsqueeze : 1 x 5 사이즈를 5 x 1 사이즈로 변경
# rand : 5 x 1 사이즈의 랜덤 값 생성
x = torch.FloatTensor(range(5)).unsqueeze(1)
y = 2*x + torch.rand(5, 1)

In [18]:
class LinearRegression(nn.Module) :
  def __init__(self) :
    # 부모 클래스의 init 함수 가져옴
    # nn.Linear(in_features, out_features, bias ... )
    super().__init__()
    self.fc = nn.Linear(1, 1, bias=True)

  def forward(self, x) :
    y = self.fc(x)
    return y

In [19]:
model = LinearRegression()
learning_rate = 1e-3
loss_func = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
loss_stack = []
for epoch in range(1000) :
  optimizer.zero_grad()

  y_hat = model(x)
  loss = loss_func(y_hat, y)

  loss.backward()
  optimizer.step()
  loss_stack.append(loss.item())

  if epoch % 100 == 0 :
    print(f'Epoch {epoch}:{loss.item()}')

Epoch 0:57.881187438964844
Epoch 100:4.05596399307251
Epoch 200:0.4203651547431946
Epoch 300:0.16354668140411377
Epoch 400:0.13545556366443634
Epoch 500:0.1239723339676857
Epoch 600:0.1146862655878067
Epoch 700:0.10650626569986343
Epoch 800:0.09925109148025513
Epoch 900:0.09281278401613235


In [22]:
# 예측은 학습이 아니므로, gradient 계산을 비활성화
with torch.no_grad():
  y_hat = model(x)

### Non-Linear Regression

In [23]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn, optim
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [25]:
df = pd.read_csv('./sample_data/california_housing_train.csv', index_col=[0])

In [30]:
X = df.drop('median_income', axis=1).to_numpy()
Y = df['median_income'].to_numpy().reshape((-1, 1))

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

In [32]:
class TensorData(Dataset) :
  def __init__(self, x_data, y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.FloatTensor(y_data)
    self.len = self.y_data.shape[0]

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

In [43]:
trainsets = TensorData(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=32, shuffle=True)

testsets = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(testsets, batch_size=32, shuffle=False)

In [45]:
class Regressor(nn.Module):
  # 신경망 선언
  def __init__(self) :
    super().__init__()
    self.fc1 = nn.Linear(7, 50, bias=True)
    self.fc2 = nn.Linear(50, 30, bias=True)
    self.fc3 = nn.Linear(30, 1, bias=True)
    self.dropout = nn.Dropout(0.5)

  # 연산 구조
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.dropout(F.relu(self.fc2(x)))
    x = F.relu(self.fc3(x))
    return x

In [46]:
model = Regressor()
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-7)

In [ ]:
loss_ = []
n = len(trainloader)

# 에포크 반복
for epoch in range(400) :
  running_loss = 0.0

  # 데이터 로더의 배치를 돌면서 학습
  for data in trainloader:
    inputs, values = data

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = loss_func(outputs, values)
    loss.backward()
    optimizer.step()

    # 학습할때마다 loss 를 더해 한 에포크의 전체 loss 를 계산
    running_loss += loss.item()

  loss_.append(running_loss/n)

### Model Evaluation - RMSE

In [48]:
def evaluation(dataloader):
  pred = torch.tensor([], dtype=torch.float)
  actual = torch.tensor([], dtype=torch.float)

  with torch.no_grad():
    model.eval()

    for data in dataloader:
      inputs, values = data
      outputs = model(inputs)
      pred = torch.cat((pred, outputs), 0)
      actual = torch.cat((actual, values), 0)

  pred = pred.numpy()
  actual = actual.numpy()
  rmse = np.sqrt(mean_squared_error(pred, actual))

  return rmse

In [51]:
train_rmse = evaluation(trainloader)
test_rmse = evaluation(testloader)
print(f'train_rmse : {train_rmse}')
print(f'train_rmse : {test_rmse}')

train_rmse : 4.298290252685547
train_rmse : 4.3555426597595215
